# **Initiation**

In [1]:
# %pip install bs4
# %pip install html2text
# %pip install clean-text
# %pip install undetected-chromedriver requests
# %pip install selenium["4.18.1"]

# **Imports**

In [2]:
import pandas as pd
import bs4
from urllib.parse import urlparse, parse_qs
import urllib.parse
from selenium import webdriver as wd2
import undetected_chromedriver as webdriver
from cleantext import clean
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from itertools import product
import html2text
from selenium.webdriver.support import expected_conditions as EC

pd.set_option('display.max_columns', None)
# from umap import UMAP
import matplotlib.ticker as plticker
import matplotlib.pyplot as plt
import sweetviz as sv
from tqdm.notebook import tqdm
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering, DBSCAN, OPTICS, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import numpy as np
import requests
from IPython.core.display import display, HTML
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# import accelerate
# from g4f.client import Client
import bs4
# import nltk
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize
# import nltk
import re
import json
# from googlesearch import search
from time import sleep
from tqdm import tqdm
from typing import List

# nltk.download('punkt')

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/mike/6semak/hack_omgtu/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_372108/2742106240.py:27: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# **Values**

In [43]:
n_of_sources = 100
std_header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}


In [20]:
driver_path = './chromedriver'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--use_subprocess")
chrome_options.add_argument("javascript.enabled")
browser = webdriver.Chrome(driver_executable_path='./chromedriver', options=chrome_options)

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


In [21]:
wait = WebDriverWait(browser, 20)

# Functions

In [22]:
def selenium_google_search_source(text: str, browser, border: int=20, body='rcnt'): # 'rcnt'
    t = urllib.parse.quote(text, safe='')
    v= f"https://www.google.com/search?channel=fs&client=ubuntu-sn&q={t}"
    browser.get(f"https://www.google.com/search?channel=fs&client=ubuntu-sn&q={t}")
    x = []
    for i in browser.find_element(by=By.ID, value=body).find_elements(by=By.TAG_NAME, value='a'):
        x.append(i.get_attribute('href'))
    return x[:border]


In [59]:
def selenium_source_link_to_html(links: List[str], browser):
    html_source = []
    bad_res = []
    for k, i in enumerate(links):
        try:
            browser.get(i)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            html_source.append(browser.page_source)
        except Exception as e:
            if type(e) is KeyboardInterrupt:
                break
            print('*', end='')
            bad_res.append(i)
        print(k, end=', ')
    return html_source, bad_res


def req_source_link_to_html(links: List[str], browser):
    html_source = []
    bad_res = []
    for k, i in enumerate(links):
        try:
            r = requests.get(i, headers=std_header)
            if r.status_code == 200:
                html_source.append(r.text)
        except Exception as e:
            if type(e) is KeyboardInterrupt:
                break
            # print('*', end='')
            bad_res.append(i)
        # print(k, end=', ')
    return html_source, bad_res



def req_source_link_to_html_saving(links: List[str], req_id, browser):
    html_source = []
    bad_res = []
    for k, i in enumerate(links):
        try:
            r = requests.get(i, headers=std_header, timeout=40)
            if r.status_code == 200:
                with open(f'./html_pages/{req_id}_{k}.html', 'w') as f:
                    f.write(r.text)
        except Exception as e:
            if type(e) is KeyboardInterrupt:
                break
            # print('*', end='')
            bad_res.append(i)
        # print(k, end=', ')
    return html_source, bad_res

In [24]:
def parse_requests(search_text: list[str], border=50):
    bad_res = []
    result_search = dict()
    for i, j in enumerate(search_text):
        try:
            sleep(1)
            x = selenium_google_search_source(j, browser=browser, border=border)
            result_search[j] = dict(enumerate(filter(lambda a: a != None, x)))
        except Exception as e:
            if type(e) is KeyboardInterrupt:
                break
            print('*', end='')
            bad_res.append(j)
        print(i, end=', ')

    return result_search, bad_res

# **Parse**

In [8]:
search_text = [
    'Agile',
    'Algorithm',
    'API',
    'Bug',
    'Compilation',
    'Constant',
    'Database',
    'Deploy',
    'Debugging',
    'Git',
    'Legacy code',
    'Library',
    'Object-Oriented Programming',
    'Develop site',
    'buy site',
    'buy hosting',
    'order develop site',
    'order web site develop',
    'order it',
    'order backend develop',
    'order site'
]

In [9]:
result_search, bad_res = parse_requests(search_text)

0, 1, *2, 

In [ ]:
x = [len(i) for i in result_search.values()]
max(x), min(x)

(50, 17)

In [ ]:
with open('google_search_res.json', 'w') as f:
    json.dump(result_search, f)


In [ ]:
result_search

{'Agile': {0: 'https://www.atlassian.com/ru/agile',
  1: 'https://www.atlassian.com/ru/agile/scrum/agile-vs-scrum',
  2: 'https://www.atlassian.com/ru/agile/project-management',
  3: 'https://www.atlassian.com/ru/agile/agile-at-scale/long-term-agile-planning',
  4: 'https://www.atlassian.com/ru/agile/scrum',
  5: 'https://www.atlassian.com/ru/agile#:~:text=Agile%20%E2%80%94%20%D1%8D%D1%82%D0%BE%20%D0%B3%D1%80%D1%83%D0%BF%D0%BF%D0%B0%20%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B9%2C%20%D0%B2,%D0%B8%D1%82%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8%20%D0%B8%D0%BB%D0%B8%20%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9%20%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%80%20%D0%BA%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4%D1%8B.',
  6: 'https://www.google.com/search?client=ubuntu-sn&hs=32B&sca_esv=e472bba1732e8ddb&sca_upv=1&channel=fs&q=%D0%A7%D1%82%D0%BE+%D1%82%D0%B0%D0%BA%D0%BE%D0%B5+Agile+%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%8B%D0%BC%D0%B8+%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D0%BC%D0%B8%

# **Parse google 2**

In [12]:
search_text_2 = [
    'разработка программных продуктов на заказ',
    'разработка приложения и веб-сервиса с нуля',
    'сопровождение существующего вебсервиса',
    "сопровождение вебсервиса",
    "разработка приложения",
    "разработка веб-сервиса",
    "доработка веб-сервиresult_searchса",
    "доработка приложения",
    "развёртывание приложение на заказ",
    "развёртывание сайта на заказ",
    "заказать развёртывание сайта",
    "заказать разработку веб-сервиса",
]

cities = [
    'омск',
    "москва",
    "томск",
    "сбп",
    "новосибирск",
    "казань",
    "екб"
]




In [25]:
search_text_and_city = list(map(lambda a: a[0] + ' ' + a[1], product(search_text_2, cities)))

In [27]:
result_search, bad_res = parse_requests(search_text=search_text_and_city, border=50)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 

In [34]:
x = [len(i) for i in result_search.values()]
max(x), min(x)

(50, 11)

In [35]:
with open('google_search_res_2.json', 'w') as f:
    json.dump(result_search, f)


In [36]:
result_search

{'разработка программных продуктов на заказ омск': {0: 'https://hwdtech.ru/',
  1: 'https://wadline.ru/software/ru/omsk',
  2: 'https://alakris.ru/omsk/razrabotka-programmnogo-obespecheniya.html',
  3: 'https://2gis.ru/omsk/search/%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0%20%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%BD%D0%BE%D0%B3%D0%BE%20%D0%BE%D0%B1%D0%B5%D1%81%D0%BF%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D1%8F/rubricId/112624',
  4: 'https://omsk.regtorg.ru/goods/razrabotka-programmnogo-obespecheniya-po/',
  5: 'https://bellintegrator.ru/razrabotka-po-na-zakaz',
  6: 'https://www.iterus.ru/',
  7: 'https://www.google.com/search?client=ubuntu-sn&hs=fhC&sca_esv=e472bba1732e8ddb&sca_upv=1&channel=fs&q=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0+%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%BD%D1%8B%D1%85+%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82%D0%BE%D0%B2+%D0%BD%D0%B0+%D0%B7%D0%B0%D0%BA%D0%B0%D0%B7+%22%D0%BE%D0%BC%D1%81%D0%BA%22&sa=X&ved=2ah

# **Get site in htmls code**

In [55]:
import uuid

In [60]:
with open('google_search_res.json', 'r') as f:
    s = json.load(f)

html_source = dict()
s_uuid_pairs = dict()


for i in tqdm(s):
    x = str(uuid.uuid4())
    s_uuid_pairs[i] = x
    html_source[i], _ = req_source_link_to_html_saving(list(s[i].values())[:10], x, browser)
    # print()

100%|██████████| 21/21 [12:15<00:00, 35.01s/it]


In [62]:
with open('small_link_source_uuid_pairs.json', 'w') as f:
    json.dump(s_uuid_pairs, f)


In [64]:
with open('google_search_res_2.json', 'r') as f:
    s = json.load(f)

html_source_2 = dict()
s_uuid_pairs = dict()

for i in s:
    x = str(uuid.uuid4())
    s_uuid_pairs[i] = x
    html_source_2[i], _ = req_source_link_to_html_saving(list(s[i].values())[:10], x, browser)

In [65]:
with open('small_link_source_uuid_pairs_2.json', 'w') as f:
    json.dump(s_uuid_pairs, f)


# **Get site in img**